Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [95]:
%reload_ext sql
%sql sqlite:///football.db
    

'Connected: None@football.db'

In [96]:
import sqlite3

In [109]:
%%sql
pragma foreign_key=ON;

drop table if exists positions;
create table positions(id integer primary key, Name text);

drop table if exists coaches;
create table coaches(id integer primary key,Name text);

drop table if exists teams;
create table teams(id integer primary key,Name text,Location text,Coach_id integer, foreign key(Coach_id) references coaches(id));

drop table if exists players;
create table players(id integer primary key,Name text, Age integer ,Position_id integer, team_id integer, Start_date date, foreign key(team_id) references teams (id), foreign key (position_id) references positions(id));

drop table if exists PlayersArchive;
create table PlayersArchive(player_id integer, team_id integer, Start_date date, end_date date, foreign key(player_id)  references players(id), foreign key (team_id) references teams(id), primary key(player_id,team_id));







Done.
(sqlite3.OperationalError) database is locked [SQL: 'drop table if exists positions;']


# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [110]:
%%sql
Insert into positions values(15,'вратарь');
Insert into positions values(20,'защитник');
Insert into positions values(36,'полузащитник');
Insert into positions values(27,'нападающий');



(sqlite3.OperationalError) database is locked [SQL: "Insert into positions values(15,'вратарь');"]


In [112]:
%%sql 
select *
from positions

Done.


id,Name
15,вратарь
20,защитник
27,нападающий
36,полузащитник


# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [114]:
db=sqlite3.connect('football.db')
def check_coach(name1): 
    cur1=db.cursor() 
    cur1.execute(''' select id from coaches where name=?''', (name1,)) 
    try: 
        co_id=cur1.fetchone()[0]
        print('Такой тренер уже существует')
        return -1
    except TypeError:
        print ('Добавлен')
        return 1
    
    
def insert_coach(name_coach):
    coach_id=check_coach(name_coach) 
    if coach_id==-1:
        return -1 
    else: 
        cur2=db.cursor() 
        cur2.execute(''' select coalesce(max(id)+1,1) as new_id from coaches ''')
        new_id=cur2.fetchone()[0]
        cur2.execute( ''' insert into coaches(id,name) values (?,?) ''',(new_id,name_coach,))
db.commit()

In [134]:
insert_coach('Alfred')
insert_coach('Anthony')

Добавлен
Добавлен


(sqlite3.OperationalError) database is locked [SQL: 'select * from coaches']


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [138]:
def insert_team(team_name,i_loc,i_coach):
    cur3=db.cursor()
    cur3.execute(''' select coalesce(max(id)+1,1) as new_id from teams ''')
    new_id=cur3.fetchone()[0]
    cur3.execute( ''' insert into teams(id, name,location,coach_id) values (?,?,?,?) ''',(new_id,team_name,i_loc,i_coach))
db.commit()
    

OperationalError: database is locked

In [ ]:
insert_team('Сборная Канады ','Америка','Mike')
insert_team('Сборная России ','Россия','Alex')


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [142]:
db=sqlite3.connect('football.db')
def check_team (name1): 
    cur1=db.cursor() 
    cur1.execute(''' select id from teams where name=?''', (name1,)) 
    try: 
        t_id=cur1.fetchone()[0]
        return t_id
    except TypeError:
        print ('Нет такой команды')
        return -1
    
    
def update_teams_loc(name_team,i_loc):
    team_id=check_team(name_team) 
    if team_id==-1:
        return -1 
    else: 
        cur2=db.cursor() 
        cur2.execute(''' update teams set location=? where name=? ''', (i_loc,name_team,))
db.commit()

In [144]:
update_teams_loc('Сборная России','РФ')

OperationalError: database is locked

# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.